# Traffic Volume Modeling Playbook

**Goal:** walk through data ingestion, exploratory analysis, engineered-feature modeling, and AutoML benchmarking for the Kaggle traffic challenge.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Set plot style
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Load Data
train_df = pd.read_csv('TRAIN.csv', parse_dates=['date_time'])
kaggle_df = pd.read_csv('KAGGLE.csv', parse_dates=['date_time'])

print("Train Shape:", train_df.shape)
print("Kaggle Shape:", kaggle_df.shape)
display(train_df.head())

Train Shape: (5000, 10)
Kaggle Shape: (6572, 10)


,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume,ID
0,NaN,38.0,0.0,0.0,1,Clear,sky is clear,2018-03-27 22:00:00,2058,1
1,NaN,54.8,0.0,0.0,90,Mist,mist,2017-04-25 09:00:00,5217,2
2,NaN,1.0,0.0,0.0,1,Clear,sky is clear,2018-02-04 14:00:00,3686,3
3,NaN,35.5,0.0,0.0,1,Mist,mist,2018-04-12 08:00:00,6198,4
4,NaN,46.1,0.0,0.0,90,Rain,light rain,2013-04-22 20:00:00,2063,5


# Final Blend Pipeline (Self-Contained)

## Required Files
**Input:**
- `TRAIN.csv` - Raw training data with columns: `date_time`, `weather_description`, `temp`, `clouds_all`, `traffic_volume`
- `KAGGLE.csv` - Raw test data with columns: `ID`, `date_time`, `weather_description`, `temp`, `clouds_all`
- `prior_best_submission/*.csv` - (Optional) Your best prior submission for blending

## Output Files
- `artifacts_hgb_engineered_10fold/bagged_ensemble_engineered_10fold.csv` - Raw 10-fold predictions
- `artifacts_hgb_engineered_10fold/bagged_ensemble_engineered_10fold_bias_corrected.csv` - Bias-corrected predictions
- `artifacts_hgb_engineered_10fold/bagged_ensemble_engineered_10fold_metrics.json` - CV metrics
- `blended_263_hgb10_bias_corrected.csv` - **Final blended submission**

## Pipeline Steps
1. Load raw data (`TRAIN.csv`, `KAGGLE.csv`)
2. Engineer features (hour, dayofweek, year, dayofyear, weather_final, temp, clouds_all)
3. Train 10-fold HGB ensemble
4. Apply hour-of-week bias correction
5. Blend with prior best submission (98% prior + 2% new)

In [ ]:
# === FINAL BLEND PIPELINE (SELF-CONTAINED) ===
# All-in-one cell: data loading, feature engineering, model training, blending, and submission
from __future__ import annotations

import inspect
import json
from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

# =============================================================================
# CONFIGURATION
# =============================================================================
# Input data files
RAW_TRAIN_PATH = Path("TRAIN.csv")
RAW_KAGGLE_PATH = Path("KAGGLE.csv")

# Output paths
ARTIFACTS = Path.cwd() / "artifacts_hgb_engineered_10fold"
ARTIFACTS.mkdir(exist_ok=True)
SUBMISSION_PATH = ARTIFACTS / "bagged_ensemble_engineered_10fold.csv"
BIAS_CORRECTED_PATH = ARTIFACTS / "bagged_ensemble_engineered_10fold_bias_corrected.csv"
METRICS_PATH = ARTIFACTS / "bagged_ensemble_engineered_10fold_metrics.json"

# Prior best submission for blending (place your best CSV here)
PRIOR_BEST_DIR = Path.cwd() / "prior_best_submission"
PRIOR_BEST_DIR.mkdir(exist_ok=True)

# Model hyperparameters
BASE_PARAMS = dict(
    learning_rate=0.03,
    max_iter=3000,
    max_depth=18,
    max_leaf_nodes=84,
    min_samples_leaf=15,
    max_bins=160,
    l2_regularization=0.10,
    early_stopping=False,
    scoring="neg_root_mean_squared_error",
)

# Ensemble settings
N_FOLDS = 10
KF_RANDOM_STATE = 2025

# Blend weight (alpha for prior best, 1-alpha for new model)
BLEND_ALPHA = 0.98

# Features used by the model
STABLE_FEATURES = [
    "hour",
    "dayofweek",
    "year",
    "dayofyear",
    "weather_final",
    "temp",
    "clouds_all",
]

# =============================================================================
# FEATURE ENGINEERING (SELF-CONTAINED)
# =============================================================================
def create_weather_final(df: pd.DataFrame) -> np.ndarray:
    """Create validated 8-level weather feature from raw weather_description."""
    desc = df["weather_description"].str.lower().str.strip()
    
    conditions = [
        desc.isin(["sky is clear", "overcast clouds"]),
        desc.isin(["few clouds", "broken clouds", "scattered clouds", "haze"]),
        desc.isin(["mist", "fog"]),
        desc.isin([
            "light rain", "drizzle", "light intensity drizzle",
            "light rain and snow", "light intensity shower rain",
        ]),
        desc.isin([
            "moderate rain", "heavy intensity rain", "freezing rain",
            "heavy intensity drizzle", "shower drizzle", "proximity shower rain",
        ]),
        desc.isin(["light snow", "light shower snow"]),
        desc.isin(["snow", "heavy snow", "sleet", "shower snow"]),
        desc.str.contains("thunderstorm", na=False),
    ]
    
    choices = [
        "Best_Conditions",
        "Cloudy_Hazy",
        "Low_Viz",
        "Rain_Light",
        "Rain_ModHeavy",
        "Snow_Light",
        "Snow_ModHeavy",
        "Thunderstorm",
    ]
    
    return np.select(conditions, choices, default="Other")


def engineer_features(df: pd.DataFrame) -> pd.DataFrame:
    """Engineer all required features for the HGB model."""
    data = df.copy()
    
    # Parse datetime
    dt = pd.to_datetime(data["date_time"], utc=False, errors="coerce")
    
    # Time features
    data["hour"] = dt.dt.hour
    data["dayofweek"] = dt.dt.dayofweek
    data["year"] = dt.dt.year
    data["dayofyear"] = dt.dt.dayofyear
    
    # Weather feature
    data["weather_final"] = create_weather_final(data)
    
    # Keep original temp and clouds_all (should already exist)
    # If temp is in Kelvin, convert to Fahrenheit
    if "temp" in data.columns:
        if data["temp"].mean() > 200:  # likely Kelvin
            data["temp"] = (data["temp"] - 273.15) * 9/5 + 32
    
    return data


def _ensure_datetime(series: pd.Series) -> pd.Series:
    """Ensure series is datetime type."""
    if series.dtype.kind == "M":
        return series
    return pd.to_datetime(series, utc=False, errors="coerce")


def build_engineered_frame(df_in: pd.DataFrame) -> pd.DataFrame:
    """Run engineer_features and attach auxiliary fields needed downstream."""
    engineered = engineer_features(df_in)
    frame = engineered.copy()
    
    frame["dayofweek"] = frame["dayofweek"].astype("category")
    frame["weather_final"] = frame["weather_final"].astype("category")
    
    if "traffic_volume" in df_in.columns:
        traffic = df_in["traffic_volume"].astype(float)
        frame["traffic_volume"] = traffic.values
        frame["traffic_volume_log"] = np.log1p(traffic).values
    
    if "ID" in df_in.columns:
        frame["ID"] = df_in["ID"].values
    
    if "date_time" in df_in.columns:
        dt = _ensure_datetime(df_in["date_time"])
        frame["date_time"] = dt
        frame["time_index"] = (dt - dt.min()).dt.total_seconds() / 3600.0
    else:
        frame["time_index"] = np.arange(len(frame), dtype=float)
    
    frame["dayofweek_numeric"] = frame["dayofweek"].astype(str).astype(int)
    frame["day_of_week"] = frame["dayofweek_numeric"]
    frame["hour_of_week"] = frame["dayofweek_numeric"] * 24 + frame["hour"]
    
    return frame.sort_values("time_index").reset_index(drop=True)


def build_X_y(train_eng: pd.DataFrame):
    """Split engineered train frame into X, y using the locked feature set."""
    if "traffic_volume" not in train_eng.columns:
        raise ValueError("Expected column 'traffic_volume' in engineered train frame.")
    
    X = train_eng[STABLE_FEATURES].copy()
    y = train_eng["traffic_volume"].astype(float)
    
    return X, y


def build_kaggle_X(kaggle_eng: pd.DataFrame, train_X_columns: list[str]):
    """Build Kaggle feature frame matching the columns used in training."""
    Xk = kaggle_eng[STABLE_FEATURES].copy()
    
    for col in train_X_columns:
        if col not in Xk.columns:
            Xk[col] = 0.0
    
    return Xk[train_X_columns]


# =============================================================================
# PIPELINE BUILDER
# =============================================================================
def build_pipeline(features: pd.DataFrame) -> Pipeline:
    """Create preprocessing + HGB model pipeline."""
    categorical_cols = features.select_dtypes(include=["object", "category"]).columns.tolist()
    numeric_cols = [col for col in features.columns if col not in categorical_cols]
    
    onehot_kwargs = {"handle_unknown": "ignore"}
    if "sparse_output" in inspect.signature(OneHotEncoder.__init__).parameters:
        onehot_kwargs["sparse_output"] = False
    else:
        onehot_kwargs["sparse"] = False
    
    transformer = ColumnTransformer(
        transformers=[
            ("categorical", OneHotEncoder(**onehot_kwargs), categorical_cols),
            ("numeric", "passthrough", numeric_cols),
        ]
    )
    
    model = HistGradientBoostingRegressor(**BASE_PARAMS)
    
    return Pipeline([
        ("preprocessor", transformer),
        ("model", model),
    ])


# =============================================================================
# BIAS CORRECTION
# =============================================================================
def apply_bias_correction(
    train_eng: pd.DataFrame,
    kaggle_eng: pd.DataFrame,
    oof_preds: np.ndarray,
    kaggle_preds: np.ndarray,
) -> np.ndarray:
    """Apply hour-of-week bias correction to Kaggle predictions."""
    y_true = train_eng["traffic_volume"].values
    residuals = y_true - oof_preds
    
    # Compute mean residual per hour_of_week
    train_eng = train_eng.copy()
    train_eng["residual"] = residuals
    hw_bias = train_eng.groupby("hour_of_week")["residual"].mean()
    
    # Apply correction to Kaggle predictions
    kaggle_hw = kaggle_eng["hour_of_week"].values
    corrections = pd.Series(kaggle_hw).map(hw_bias).fillna(0.0).values
    
    corrected = kaggle_preds + corrections
    return np.clip(corrected, 0, None)


# =============================================================================
# MAIN PIPELINE
# =============================================================================
def run_final_blend_pipeline():
    """Execute the complete pipeline: train, predict, correct bias, and blend."""
    
    print("=" * 60)
    print("FINAL BLEND PIPELINE")
    print("=" * 60)
    
    # --- Load Data ---
    print("\n[1/6] Loading data...")
    train_df = pd.read_csv(RAW_TRAIN_PATH, parse_dates=["date_time"])
    kaggle_df = pd.read_csv(RAW_KAGGLE_PATH, parse_dates=["date_time"])
    print(f"  Train: {train_df.shape[0]:,} rows")
    print(f"  Kaggle: {kaggle_df.shape[0]:,} rows")
    
    # --- Engineer Features ---
    print("\n[2/6] Engineering features...")
    train_eng = build_engineered_frame(train_df)
    kaggle_eng = build_engineered_frame(kaggle_df)
    
    X, y = build_X_y(train_eng)
    train_X_cols = X.columns.tolist()
    X_kaggle = build_kaggle_X(kaggle_eng, train_X_cols)
    
    print(f"  Training features: {X.shape}")
    print(f"  Kaggle features: {X_kaggle.shape}")
    print(f"  Features: {train_X_cols}")
    
    # --- Train 10-Fold Ensemble ---
    print(f"\n[3/6] Training {N_FOLDS}-fold HGB ensemble...")
    kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=KF_RANDOM_STATE)
    
    oof_predictions = np.zeros(len(X), dtype=float)
    prediction_counts = np.zeros(len(X), dtype=float)
    kaggle_predictions = np.zeros(X_kaggle.shape[0], dtype=float)
    fold_metrics = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y), start=1):
        print(f"  Fold {fold}/{N_FOLDS}...", end=" ")
        
        X_train_fold = X.iloc[train_idx]
        y_train_fold = y.iloc[train_idx]
        X_val_fold = X.iloc[val_idx]
        y_val_fold = y.iloc[val_idx]
        
        pipeline = build_pipeline(X_train_fold)
        pipeline.fit(X_train_fold, y_train_fold)
        
        val_preds = pipeline.predict(X_val_fold)
        oof_predictions[val_idx] += val_preds
        prediction_counts[val_idx] += 1
        
        fold_rmse = mean_squared_error(y_val_fold, val_preds, squared=False)
        fold_metrics.append({"fold": fold, "rmse": float(fold_rmse)})
        print(f"RMSE: {fold_rmse:.3f}")
        
        kaggle_predictions += pipeline.predict(X_kaggle) / N_FOLDS
    
    prediction_counts[prediction_counts == 0] = 1.0
    oof_predictions /= prediction_counts
    
    overall_oof_rmse = float(mean_squared_error(y, oof_predictions, squared=False))
    print(f"\n  Overall OOF RMSE: {overall_oof_rmse:.3f}")
    
    # Save artifacts
    np.save(ARTIFACTS / "oof_preds.npy", oof_predictions)
    np.save(ARTIFACTS / "y_true.npy", y.to_numpy())
    
    metrics = {
        "overall_oof_rmse": overall_oof_rmse,
        "base_model_params": BASE_PARAMS,
        "n_folds": N_FOLDS,
        "fold_metrics": fold_metrics,
        "feature_columns": train_X_cols,
    }
    METRICS_PATH.write_text(json.dumps(metrics, indent=2))
    
    # Save raw submission
    raw_submission = pd.DataFrame({
        "ID": kaggle_eng.get("ID", pd.Series(np.arange(len(X_kaggle)))).values,
        "traffic_volume": kaggle_predictions,
    })
    raw_submission.to_csv(SUBMISSION_PATH, index=False)
    print(f"\n[4/6] Raw submission saved: {SUBMISSION_PATH.name}")
    
    # --- Apply Bias Correction ---
    print("\n[5/6] Applying bias correction...")
    kaggle_predictions_corrected = apply_bias_correction(
        train_eng, kaggle_eng, oof_predictions, kaggle_predictions
    )
    
    corrected_submission = pd.DataFrame({
        "ID": kaggle_eng.get("ID", pd.Series(np.arange(len(X_kaggle)))).values,
        "traffic_volume": kaggle_predictions_corrected,
    })
    corrected_submission.to_csv(BIAS_CORRECTED_PATH, index=False)
    print(f"  Bias-corrected submission saved: {BIAS_CORRECTED_PATH.name}")
    
    # --- Blend with Prior Best ---
    print("\n[6/6] Blending with prior best submission...")
    
    # Find prior best CSV
    prior_best_path = None
    if PRIOR_BEST_DIR.exists():
        csv_files = list(PRIOR_BEST_DIR.glob("*.csv"))
        if csv_files:
            prior_best_path = max(csv_files, key=lambda p: p.stat().st_mtime)
    
    if prior_best_path is None:
        print(f"  WARNING: No prior best CSV found in {PRIOR_BEST_DIR}")
        print(f"  Place your best submission CSV in that folder to enable blending.")
        print(f"  Using bias-corrected submission as final output.")
        final_submission = corrected_submission.copy()
        final_path = BIAS_CORRECTED_PATH.with_name("final_submission.csv")
    else:
        print(f"  Prior best: {prior_best_path.name}")
        prior_best = pd.read_csv(prior_best_path)
        
        merged = prior_best.merge(
            corrected_submission,
            on="ID",
            suffixes=("_prior", "_new")
        )
        
        y_prior = merged["traffic_volume_prior"].to_numpy()
        y_new = merged["traffic_volume_new"].to_numpy()
        y_blend = BLEND_ALPHA * y_prior + (1 - BLEND_ALPHA) * y_new
        
        final_submission = pd.DataFrame({
            "ID": merged["ID"],
            "traffic_volume": y_blend,
        })
        
        final_path = Path.cwd() / "blended_263_hgb10_bias_corrected.csv"
        print(f"  Blend: {BLEND_ALPHA:.0%} prior + {1-BLEND_ALPHA:.0%} new")
    
    final_submission.to_csv(final_path, index=False)
    print(f"  Final submission saved: {final_path.name}")
    
    print("\n" + "=" * 60)
    print("PIPELINE COMPLETE")
    print("=" * 60)
    
    return {
        "train_eng": train_eng,
        "kaggle_eng": kaggle_eng,
        "oof_preds": oof_predictions,
        "kaggle_preds": kaggle_predictions,
        "kaggle_preds_corrected": kaggle_predictions_corrected,
        "final_submission": final_submission,
        "metrics": metrics,
    }


# Run the pipeline
results = run_final_blend_pipeline()

# Plots for HW_9.docx Report

This cell generates all visualizations required for the deliverables:
1. **Hour × Day-of-Week Interaction Heatmap** - Shows traffic patterns across the week
2. **Weather vs Traffic Volume Boxplot** - Impact of weather_final categories
3. **Temperature vs Traffic Volume** - Relationship with temperature
4. **Holiday Impact Visualization** - Traffic reduction on holidays
5. **Model Pipeline Diagram** - How predictions are assembled